To fix list:
- The Year scraping bit isn't working past 2003. It's getting crowd attendance.
- Need to grab brownlow votes as well

In [59]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import re

first_game = 1285
last_game = 2573

dfs = []
for match_id in range(first_game, last_game+1):
    print(match_id)
    url = f"https://www.footywire.com/afl/footy/ft_match_statistics?mid={match_id}"
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")

    # Find the table element with the match statistics
    match_stats_table = soup.find("table", {"id": "match-statistics-div"})
    if match_stats_table is None:
        print(f"No match statistics div found for match ID {match_id}")
        continue

    # Find the round number and year
    year_round = soup.find('td', {"class": "lnorm", "height":"22"})
    round = year_round.text[0:year_round.text.find(',')]
    year = year_round.text[-4:]

    if 'final' in round.lower():
        print(f"There is 'final' in this round: {round}")
        continue

    # Find the rows with player statistics
    player_stats_rows = match_stats_table.find_all("tr", {"class": ["darkcolor", "lightcolor"], "onmouseover": "this.className='highlightcolor';", "onmouseout": ["this.className='darkcolor';", "this.className='lightcolor';"]})

    # Extract the data from each row and create a pandas dataframe
    data = []
    for row in player_stats_rows:
        cols = row.find_all("td")
        cols = [col.text.strip() for col in cols]
        if len(cols) == 11:
            new_row = [year, round] + cols
            # print(new_row)
            data.append(new_row)

    if data:
        df = pd.DataFrame(data, columns=["Year", "Round", "Player", "K", "HB", "D", "M", "G", "B", "T", "HO", "FF", "FA"])
        df["Match ID"] = match_id

        # Get Brownlow votes
        brownlow_votes = soup.find(text=re.compile("Brownlow Votes:"))
        if brownlow_votes:
            votes = {}
            for i in range(3):
                player = brownlow_votes.find_next("a").text.strip()
                player_parts = player.split(" ")
                last_name = player_parts[-1]
                first_initial = player_parts[0][0]
                player_reversed = " ".join(player_parts[::-1])
                votes[player_reversed] = 3-i
                brownlow_votes = brownlow_votes.find_next("a")
            df["Votes"] = df["Player"].apply(lambda x: votes.get(x.split()[-1] + ' ' + x.split()[0][0], 0))
        else:
            df["Votes"] = 0

        dfs.append(df)
        print(f"Processed match ID {match_id}")

# Concatenate all dataframes into a single one
df_final = pd.concat(dfs)

# Save the data to a CSV file
df_final.to_csv("player_stats.csv", index=False)


1285
Processed match ID 1285
1286
Processed match ID 1286
1287
Processed match ID 1287
1288
Processed match ID 1288
1289
Processed match ID 1289
1290
Processed match ID 1290
1291
Processed match ID 1291
1292
Processed match ID 1292
1293
Processed match ID 1293
1294
Processed match ID 1294
1295
Processed match ID 1295
1296
Processed match ID 1296
1297
Processed match ID 1297
1298
Processed match ID 1298
1299
Processed match ID 1299
1300
Processed match ID 1300
1301
Processed match ID 1301
1302
Processed match ID 1302
1303
Processed match ID 1303
1304
Processed match ID 1304
1305
Processed match ID 1305
1306
Processed match ID 1306
1307
Processed match ID 1307
1308
Processed match ID 1308
1309
Processed match ID 1309
1310
Processed match ID 1310
1311
Processed match ID 1311
1312
Processed match ID 1312
1313
Processed match ID 1313
1314
Processed match ID 1314
1315
Processed match ID 1315
1316
Processed match ID 1316
1317
Processed match ID 1317
1318
Processed match ID 1318
1319
Processed

In [63]:
player_stats_rows

[<tr class="darkcolor" onmouseout="this.className='darkcolor';" onmouseover="this.className='highlightcolor';">
 <td align="left" height="18"><a href="pp-brisbane-lions--grant-birchall" title="Grant Birchall">Grant Birchall</a> </td>
 <td class="statdata">16</td>
 <td class="statdata">20</td>
 <td class="statdata">36</td>
 <td class="statdata">11</td>
 <td class="statdata">0</td>
 <td class="statdata">1</td>
 <td class="statdata">1</td>
 <td class="statdata">0</td>
 <td class="statdata">2</td>
 <td class="statdata">1</td>
 </tr>,
 <tr class="lightcolor" onmouseout="this.className='lightcolor';" onmouseover="this.className='highlightcolor';">
 <td align="left" height="18"><a href="pp-west-coast-eagles--sam-mitchell" title="Sam Mitchell">Sam Mitchell</a> </td>
 <td class="statdata">10</td>
 <td class="statdata">23</td>
 <td class="statdata">33</td>
 <td class="statdata">9</td>
 <td class="statdata">0</td>
 <td class="statdata">0</td>
 <td class="statdata">3</td>
 <td class="statdata">0</